In [1]:
import funcoes as fc
import numpy as np
import os
import pandas as pd
import pyautogui
import pyperclip
import time
import tkinter as tk
import undetected_chromedriver as uc
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tabulate import tabulate
from tkinter import scrolledtext
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#INSTALA O CHROME DRIVEr MAIS ATUALIZADO
# Desativa o bloqueio de pop-ups
options = webdriver.ChromeOptions()
options.add_argument("--disable-popup-blocking") 
#DEFINE O TEMPO DE EXECUÇÃO PARA CADA COMANDO DO PYAUTOGUI
pyautogui.PAUSE = 0.7
#INICIA O NAVEGADOR
# Caminho do ChromeDriver local
# chrome_driver_path = r"main\chromedriver-win64\chromedriver.exe"  # Altere para o caminho correto do seu ChromeDriver
# Configura o serviço do ChromeDriver
servico = Service(ChromeDriverManager().install())
# Inicia o navegador Chrome
navegador = webdriver.Chrome(service=servico, options=options)
navegador.maximize_window()
#ENTRA NO SEI
navegador.get('https://sei.anatel.gov.br/')


In [8]:
def get_text():
    #ENVIA TEXTO DO INPUT
    global texto_exigencia
    texto_exigencia = text_area.get("1.0", tk.END).strip()
    #FECHA A JANELA DE INPUT
    root.destroy()

#INICIALIZA A VARIAVEL LOCAL QUE RECEBERA O TEXTO DE INPUT
texto_exigencia = ""

#CRIA A JANELA DA INTERFACE PARA CAIXA DE INPUT
root = tk.Tk()
root.title("Inserir Texto de Exigência")

#INSTRUCAO DO QUE O USUARIO DEVE FAZER
label_instruction = tk.Label(root, text="Insira apenas a exigência:")
label_instruction.pack(padx=10, pady=5)

#CRIA AREA DE TEXTO COM ROLAGEM
text_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=80, height=20)
text_area.pack(padx=10, pady=10)

#CRIA BOTAO DE RECEBER O TEXTO
btn_get_text = tk.Button(root, text="Obter Texto", command=get_text)
btn_get_text.pack(pady=5)

root.bind('<Control-Return>', lambda event: get_text())

#INICIA O LOOP PRINCIPAL QUE MANTEM A JANELA ATIVA
root.mainloop()

In [12]:
import pygetwindow as gw

def muda_janela(janela: str):
    # Lista todas as janelas abertas
    windows = gw.getAllTitles()

    # Filtra janelas válidas (não-vazias)
    windows = [w for w in windows if w]

    for i in range(len(windows)):
        if janela in windows[i]:
            window_to_activate = i
    window = gw.getWindowsWithTitle(windows[window_to_activate])[0]
    window.activate()
    window.maximize()
# Alternar para uma janela específica
# window_to_activate = int(input("Digite o número da janela para ativar: "))
# window = gw.getWindowsWithTitle(windows[window_to_activate])[0]
# window.activate()


In [4]:
from pywinauto import Application

# Substitua pelo título ou caminho do programa que deseja manipular
app = Application().connect(title_re=".*Edge.*")  # Conecta ao Microsoft Edge
window = app.top_window() 
window.restore()  # Restaura a janela
window.set_focus()  # Traz a janela para o primeiro plano


<hwndwrapper.DialogWrapper - 'Nova guia - Trabalho — Microsoft​ Edge', Chrome_WidgetWin_1, 393416>

In [3]:
import subprocess

# Abre o Edge
subprocess.Popen(["C:\\Program Files (x86)\\Microsoft\\Edge\\Application\\msedge.exe"])



<Popen: returncode: None args: ['C:\\Program Files (x86)\\Microsoft\\Edge\\A...>

In [16]:
fc.vai_para_processo(navegador, "53500.002572/2025-15")
time.sleep(1)
navegador.switch_to.frame('ifrConteudoVisualizacao')
#CLICA NO ICONE DE ENVIO DE EMAIL
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/email_enviar.svg?18')]")
#clica_noelemento(navegador, By.XPATH, '//*[@id="divArvoreAcoes"]/a[10]')
time.sleep(1)
#VAI PARA A JANELA MAIS RECENTE ABERTA
navegador.switch_to.window(navegador.window_handles[-1])

In [17]:

select_element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="selTextoPadrao"]')))
select = Select(select_element)
#SELECIONA A OPCAO DE EMAIL ANATEL
select.select_by_visible_text('Processo SEI ORCN - APROVAÇÃO - DRONE')


In [25]:
navegador.switch_to.window(navegador.window_handles[0])

In [24]:
processosAssinados = '53500.000035/2025-31'
retido = 'nao'
# Interagir com o navegador para acessar a página do processo
navegador.switch_to.default_content()
navegador.find_element(By.ID, 'txtPesquisaRapida').clear()
navegador.find_element(By.ID, 'txtPesquisaRapida').send_keys(processosAssinados)
navegador.find_element(By.ID, 'txtPesquisaRapida').send_keys(Keys.ENTER)
time.sleep(1.5)

#PEGA DADOS DA DECLARACAO DE CONFORMIDADE
navegador.switch_to.frame('ifrArvore')

#CONFERE SE EXISTE A DECLARACAO DE CONFORMIDADE OU UMA PASTA
if fc.check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade", navegador):
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
    time.sleep(0.3)
elif fc.check_element_exists(By.XPATH, '//*[@id="spanPASTA1"]', navegador):
    navegador.find_element(By.XPATH, '//*[@id="spanPASTA1"]').click()
    time.sleep(1)
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
    time.sleep(0.3)

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
navegador.switch_to.frame('ifrVisualizacao')

#ARMAZENA EMAIL DO SOLICITANTE
emailSol = navegador.find_element(By.XPATH, '/html/body/div[3]/a[1]').text

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')

#VERIFICA SE EXISTE O DESPACHO DECISORIO
if fc.check_element_exists(By.PARTIAL_LINK_TEXT, "Despacho Decisório", navegador):
    #CLICA NO DESPACHO DECISORIO
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Despacho Decisório").click()
    # navegador.find_element(By.PARTIAL_LINK_TEXT, "Despacho Decisório").click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    navegador.switch_to.frame('ifrVisualizacao')
    #VERIFICA SE EXISTE ASSINATURA DO GERENTE
    if fc.check_element_exists(By.XPATH, "/html/body/div[1]/table[1]", navegador):
        navegador.switch_to.default_content()
        navegador.switch_to.frame('ifrConteudoVisualizacao')
        #CLICA NO ICONE DE EMAIL
        fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/email_enviar.svg?18')]")
        #clica_noelemento(navegador, By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]')
        # navegador.find_element(By.XPATH, "//img[@title='Enviar Documento por Correio Eletrônico']").click()
        time.sleep(0.7)
        #MUDA PARA A JANELA MAIS RECENTE
        navegador.switch_to.window(navegador.window_handles[-1])
        #time.sleep(2)
        #SELECIONA O DROPDOWN COM AS OPCOES DE EMAIL
        select_element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="selDe"]')))
        select = Select(select_element)
        #SELECIONA A OPCAO DE EMAIL ANATEL
        select.select_by_visible_text('ANATEL/E-mail de replicação <nao-responda@anatel.gov.br>')
        #INSERE EMAIL DO SOLICITANTE E CLICA NO EMAIL DO SOLICITANTE
        navegador.find_element(By.XPATH, '//*[@id="s2id_autogen1"]').send_keys(emailSol)
        time.sleep(1)
        #CLICA NO EMAIL DO SOLICITANTE
        fc.clica_noelemento(navegador, By.XPATH,'//*[@id="select2-result-label-2"]')
        # navegador.find_element(By.XPATH, '//*[@id="select2-result-label-2"]').click()
        #INSERE ASSUNTO DO EMAIL E TEXTO DO EMAIL
        if retido == "sim":
            fc.endereco_email("notificacaosei.sp@anatel.gov.br", navegador)
            fc.endereco_email("documentacao.rj@anatel.gov.br", navegador)
            fc.endereco_email("fiscalizacao1.pr@anatel.gov.br", navegador)
            navegador.find_element(By.ID, 'txtAssunto').send_keys(f'Processo SEI nº {processosAssinados} - Aprovado (teste)')
            select_element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="selTextoPadrao"]')))
            select = Select(select_element)
            #SELECIONA O TEXTO PADRAO DE DRONES RETIDOS APROVADOS
            select.select_by_visible_text('Processo SEI ORCN - APROVAÇÃO - DRONE RETIDO')
        else:
            navegador.find_element(By.ID, 'txtAssunto').send_keys(f'Processo SEI nº {processosAssinados} - Aprovado')
            select_element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="selTextoPadrao"]')))
            select = Select(select_element)
            #SELECIONA O TEXTO PADRAO DE DRONES APROVADOS
            select.select_by_visible_text('Processo SEI ORCN - APROVAÇÃO - DRONE')

In [5]:
def abrir_janela():
    janela = tk.Toplevel()
    janela.title("Nova Janela")
    janela.geometry("300x200")

    # Garante que a nova janela fique em foco e no topo
    janela.attributes('-topmost', 1)  # Mantém a janela no topo
    janela.lift()                     # Eleva a janela
    janela.focus_force()              # Força o foco na janela

    tk.Label(janela, text="Essa janela está em foco!").pack(pady=20)
    tk.Button(janela, text="Fechar", command=janela.destroy).pack()

# Janela principal
root = tk.Tk()
root.title("Janela Principal")
root.geometry("300x200")

tk.Button(root, text="Abrir Nova Janela", command=abrir_janela).pack(pady=50)

root.mainloop()

In [30]:
if processo_reaberto:
    fecha_processo()

In [14]:

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
lista_despachos = navegador.find_elements(By.PARTIAL_LINK_TEXT, 'Despacho Decisório')
for i in lista_despachos:
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrArvore')
    i.click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    navegador.switch_to.frame('ifrVisualizacao')
    time.sleep(1)
    if fc.check_element_exists(By.XPATH, "//*[contains(text(), 'Revogar o')]", navegador):
        navegador.switch_to.default_content()
        navegador.switch_to.frame('ifrConteudoVisualizacao')
        print('processo achado')
        time.sleep(1)
        #CLICA NO ICONE DE LEGO
        fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/bloco_incluir_protocolo.svg?18')]")
        
        #navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
        #SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
        time.sleep(1.5)
        navegador.switch_to.frame('ifrVisualizacao')
        select_element = navegador.find_element(By.ID, 'selBloco')
        select = Select(select_element)
        #PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
        for opcao in select.options:
            if "Despachos para Drones aprovados" in opcao.text:
                select.select_by_visible_text(opcao.text)
                break
        time.sleep(0.5)
        #CLICA NO BOTAO DE INCLUIR NO BLOCO
        fc.clica_noelemento(navegador, By.XPATH,'//*[@id="sbmIncluir"]')
        #navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()
        



processo achado


In [15]:
#VOLTA PARA PAGINA INICIAL DO PROCESSO
navegador.switch_to.default_content()
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys('53500.047484/2024-61')
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)
navegador.switch_to.frame('ifrConteudoVisualizacao')
time.sleep(1)
#ADICIONA NOTA PARA AGUARDAR ASSINATURA
#CLICA NO ICONE DE ANOTACAO
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/anotacao_cadastro.svg?18')]")
#clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[16]')
time.sleep(0.1)
#INSERE O TEXTO DA ANOTACAO
navegador.switch_to.frame('ifrVisualizacao')
navegador.find_element(By.ID, 'txaDescricao').send_keys('Pedido de Cancelamento')
#SALVA O TEXTO
navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
#DA COMO APROVADO E ANOTA NO TXT DE PROCESSOS CONFORMES

In [2]:
import funcoes as fc
fc.preenche_planilhageral('53500.000742/2025-27', 'André Jacinto Rodrigues', 'Não', 'Aprovado', '', 'Rafael Basilio Chaves')

Célula encontrada: 53500.000742/2025-27
Parece que a janela 'Google Chrome' não está visível para o programa.
Parece que a janela 'theomation' não está visível para o programa.
